# Wedge Task 2: Creating A Sample of Owners

In [2]:
import os
import re
import datetime 
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor
from random import sample

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

## Initialize connection to GBQ

In [3]:
#establish Connection to GBQ

# These first two values will be different on your machine. 
service_path = "C:\\Users\\ofano\\Documents\\MSBA Folder\\MSBA Folder\\"
service_file = 'msba-project-2022-75bb8251ef6f.json' 
gbq_proj_id = 'msba-project-2022' 

# And this should stay the same. 
private_key =service_path + service_file

In [4]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [5]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [1]:
#let's make sure it works

for item in client.list_datasets() : 
    print(item.full_dataset_id)

## Query the data set in GBQ

In [7]:
#now that we have connected to our GBQ data, we need to build a list of owners, and print them out.

owner_list = []

query = """
        SELECT DISTINCT(card_no)
        FROM `msba-project-2022.wedge_dataset.transArchive_20*`
        WHERE card_no != 3"""

results = client.query(
    query,
    location="US",
)

#list comprehension that joins on tabs and converts all items to strings.
for row in results :
    owner_list.append(row.values()[0])


In [9]:
# this section creates our random sample of owners
random.seed(10)
owner_sample = random.choices(owner_list, k=400)

## Query owner information from GBQ

In [11]:
#Now let's get all the data related to our sample of owners

query = """     
SELECT DISTINCT *
FROM `msba-project-2022.wedge_dataset.transArchive_201007_201009`
WHERE card_no in (
"""
query = query +",".join([str(element) for element in owner_sample])+")"


## Export Data

In [13]:
# use this line to write out the file to a pandas dataframe
extracted_owners = client.query(query).to_dataframe()


In [14]:
#this will write out our file, use this
extracted_owners.to_csv("output_file_task_2.csv")